In [ ]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
from math import pi
from std_msgs.msg import String
from moveit_commander.conversions import pose_to_list

#moveit initialize
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('move_group_python_interface_tutorial',
                anonymous=True)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "ur3"
group = moveit_commander.MoveGroupCommander(group_name)
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)



In [ ]:
import roslib; roslib.load_manifest('robotiq_2f_gripper_control')
from robotiq_2f_gripper_control.msg import _Robotiq2FGripper_robot_output  as outputMsg

robotiq_pub = rospy.Publisher('Robotiq2FGripperRobotOutput', 
                      outputMsg.Robotiq2FGripper_robot_output,
                      queue_size=10)

#reset
command = outputMsg.Robotiq2FGripper_robot_output()
command.rACT = 0 #reset
robotiq_pub.publish(command)
rospy.sleep(0.5)

#close
command = outputMsg.Robotiq2FGripper_robot_output();
command.rACT = 1
command.rGTO = 1
command.rSP  = 255
command.rFR  = 150
command.rPR = 255
robotiq_pub.publish(command)
rospy.sleep(1.0)

#open
command = outputMsg.Robotiq2FGripper_robot_output();
command.rACT = 1
command.rGTO = 1
command.rSP  = 255
command.rFR  = 150
command.rPR = 0
robotiq_pub.publish(command)
rospy.sleep(1.0)

In [ ]:
def wait_for_state_update(box_is_known=False, box_is_attached=False, timeout=4):
    # Copy class variables to local variables to make the web tutorials more clear.
    # In practice, you should use the class variables directly unless you have a good
    # reason not to.
    
    box_name='cube'

    ## BEGIN_SUB_TUTORIAL wait_for_scene_update
    ##
    ## Ensuring Collision Updates Are Receieved
    ## ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ## If the Python node dies before publishing a collision object update message, the message
    ## could get lost and the box will not appear. To ensure that the updates are
    ## made, we wait until we see the changes reflected in the
    ## ``get_known_object_names()`` and ``get_known_object_names()`` lists.
    ## For the purpose of this tutorial, we call this function after adding,
    ## removing, attaching or detaching an object in the planning scene. We then wait
    ## until the updates have been made or ``timeout`` seconds have passed
    start = rospy.get_time()
    seconds = rospy.get_time()
    while (seconds - start < timeout) and not rospy.is_shutdown():
        # Test if the box is in attached objects
        attached_objects = scene.get_attached_objects([box_name])
        is_attached = len(attached_objects.keys()) > 0

        # Test if the box is in the scene.
        # Note that attaching the box will remove it from known_objects
        is_known = box_name in scene.get_known_object_names()
        print scene.get_known_object_names()
        
        # Test if we are in the expected state
        if (box_is_attached == is_attached) and (box_is_known == is_known):
            return True

        # Sleep so that we give other threads time on the processor
        rospy.sleep(0.1)
        seconds = rospy.get_time()

    # If we exited the while loop without returning then we timed out
    return False
    ## END_SUB_TUTORIAL

In [ ]:
#todo cube position
import tf
import numpy as np
from tf.transformations import *

listener = tf.TransformListener()
broadcaster = tf.TransformBroadcaster()

searching = True
# while(searching):
#     try:
#         #tf.waitForTransform('/world', '/rgb_camera_link', rospy.Time(0), rospy.Duration(3.0))
#         (trans,rot) = listener.lookupTransform('/world', '/ar_marker_4', rospy.Time(0))
#         euler = list(tf.transformations.euler_from_quaternion(rot))
#         trans[2] = 0.80 #center of cubes
#         euler[0] = 0.0
#         euler[1] = 0.0
#         orientation = quaternion_from_euler(0, 0, euler[2])
#         broadcaster.sendTransform(trans, orientation, rospy.Time.now(), 'ar_marker_4_target', 'world')
        
#         searching = False
        
#     except (tf.LookupException, tf.ConnectivityException, tf.ExtrapolationException) as e:
#         pass#print e

trans = [-0.65, 0.0, 0.85]
euler = [0,0,1.5707] #dummy marker position


goal_trans = trans
goal_euler = euler
#todo sanity check

print goal_trans

In [ ]:
#attach box
box_pose = geometry_msgs.msg.PoseStamped()
box_pose.header.frame_id = "world"
box_pose.pose.orientation.w = 1.0
box_pose.pose.position.x = goal_trans[0]
box_pose.pose.position.y = goal_trans[1]
box_pose.pose.position.z = goal_trans[2]
box_name = "cube"
scene.add_box(box_name, box_pose, size=(0.06, 0.06, 0.06))

print wait_for_state_update(box_is_known=True, timeout=4)

from moveit_msgs.msg import PlanningScene
from moveit_msgs.srv import ApplyPlanningScene

msg = PlanningScene()
msg.is_diff = True
msg.name = 'ur3'
planning_scene_diff_publisher = rospy.Publisher('planning_scene',PlanningScene, queue_size=1)
planning_scene_diff_publisher.publish(msg)

In [ ]:
#goto pre grasp position
pose_goal = geometry_msgs.msg.Pose()
import numpy as np
from tf.transformations import *
# q = quaternion_from_euler(0.0, -4*np.pi/4.0, 0.0)
q = quaternion_from_euler(0.0, -4*np.pi/4.0, goal_euler[2])
pose_goal.orientation.x = q[0]
pose_goal.orientation.y = q[1]
pose_goal.orientation.z = q[2]
pose_goal.orientation.w = q[3]

# pose_goal.position.x = -0.6
# pose_goal.position.y = 0.0
# pose_goal.position.z = 1.0
pose_goal.position.x = goal_trans[0] #+ 0.1
pose_goal.position.y = goal_trans[1]
pose_goal.position.z = goal_trans[2] + 0.1

group.set_pose_target(pose_goal)

plan = group.go(wait=True)
# Calling `stop()` ensures that there is no residual movement
group.stop()

rospy.sleep(0.5)
plan = group.go(wait=True)
group.stop()

# Calling `stop()` ensures that there is no residual movement
group.stop()

# It is always good to clear your targets after planning with poses.
# Note: there is no equivalent function for clear_joint_value_targets()
group.clear_pose_targets()

In [ ]:
#cartesian path to approach

waypoints = []

scale = 0.1
wpose = group.get_current_pose().pose
wpose.position.z -= scale * 1  
# wpose.position.x -= scale * 1  
waypoints.append(copy.deepcopy(wpose))


# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0 disabling:
(plan, fraction) = group.compute_cartesian_path(
                                   waypoints,   # waypoints to follow
                                   0.01,        # eef_step
                                   0.0)         # jump_threshold
group.execute(plan, wait=True)

rospy.sleep(0.5)
group.execute(plan, wait=True)
group.clear_pose_targets()


In [ ]:
#todo grasp

#close
command = outputMsg.Robotiq2FGripper_robot_output();
command.rACT = 1
command.rGTO = 1
command.rSP  = 255
command.rFR  = 150
command.rPR = 255
robotiq_pub.publish(command)
rospy.sleep(0.5)


In [ ]:
grasping_group = 'ur3'
touch_links = robot.get_link_names(group=grasping_group)
scene.attach_box('tool0', box_name, touch_links=touch_links)

In [ ]:
#cartesian path to retreat

waypoints = []

scale = -0.1
wpose = group.get_current_pose().pose
wpose.position.z -= scale * 1  
# wpose.position.x -= scale * 0.1  
waypoints.append(copy.deepcopy(wpose))


# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0 disabling:
(plan, fraction) = group.compute_cartesian_path(
                                   waypoints,   # waypoints to follow
                                   0.01,        # eef_step
                                   0.0)         # jump_threshold
group.execute(plan, wait=True)

In [ ]:
# # goto place location
# # goup.set_path_constraints()
# constraints = moveit_msgs.msg.Constraints()
# orientation_constraint = moveit_msgs.msg.OrientationConstraint()
# q = quaternion_from_euler(0.0, -4*np.pi/4.0, 0.0)
# # The desired orientation of the robot link specified as a quaternion
# orientation_constraint.orientation.x = q[0]
# orientation_constraint.orientation.y = q[1]
# orientation_constraint.orientation.z = q[2]
# orientation_constraint.orientation.w = q[3]
# # The robot link this constraint refers to
# orientation_constraint.header.frame_id = 'base_link' 
# orientation_constraint.link_name = 'tool0'
# # optional axis-angle error tolerances specified
# orientation_constraint.absolute_x_axis_tolerance = 1.2
# orientation_constraint.absolute_y_axis_tolerance = 1.2
# orientation_constraint.absolute_z_axis_tolerance = 1.2
# # A weighting factor for this constraint (denotes relative importance to other constraints. Closer to zero means less important)
# orientation_constraint.weight = 0.5
# constraints.name = 'place constraints'
# # constraints.orientation_constraints = [orientation_constraint] 

# joint_constraint = moveit_msgs.msg.JointConstraint()
# joint_constraint.joint_name = 'elbow_joint'

# group.set_path_constraints(constraints)

In [ ]:
 # goto place location
pose_goal = geometry_msgs.msg.Pose()
import numpy as np
from tf.transformations import *
q = quaternion_from_euler(0.0, 4*np.pi/4.0, np.pi/2.0)
pose_goal.orientation.x = q[0]
pose_goal.orientation.y = q[1]
pose_goal.orientation.z = q[2]
pose_goal.orientation.w = q[3]

pose_goal.position.x = -0.1
pose_goal.position.y = -0.20
pose_goal.position.z = 0.785
group.set_pose_target(pose_goal)

plan = group.go(wait=True)
# Calling `stop()` ensures that there is no residual movement
group.stop()
# It is always good to clear your targets after planning with poses.
# Note: there is no equivalent function for clear_joint_value_targets()
group.clear_pose_targets()
group.clear_path_constraints()

In [ ]:
#cartesian path to release

waypoints = []

scale = 0.1
wpose = group.get_current_pose().pose
wpose.position.z -= scale * 1  
# wpose.position.x -= scale * 0.1  
waypoints.append(copy.deepcopy(wpose))


# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0 disabling:
(plan, fraction) = group.compute_cartesian_path(
                                   waypoints,   # waypoints to follow
                                   0.01,        # eef_step
                                   0.0)         # jump_threshold
group.execute(plan, wait=True)


In [ ]:
#release

#open
command = outputMsg.Robotiq2FGripper_robot_output();
command.rACT = 1
command.rGTO = 1
command.rSP  = 255
command.rFR  = 150
command.rPR = 0
robotiq_pub.publish(command)
rospy.sleep(0.5)


In [ ]:
#release cube
scene.remove_attached_object('tool0', name=box_name)
scene.remove_world_object('cube')

In [ ]:
#cartesian path to retreat

waypoints = []

scale = 0.1
wpose = group.get_current_pose().pose
wpose.position.z += scale * 1
# wpose.position.x -= scale * 0.1  
waypoints.append(copy.deepcopy(wpose))


# We want the Cartesian path to be interpolated at a resolution of 1 cm
# which is why we will specify 0.01 as the eef_step in Cartesian
# translation.  We will disable the jump threshold by setting it to 0.0 disabling:
(plan, fraction) = group.compute_cartesian_path(
                                   waypoints,   # waypoints to follow
                                   0.01,        # eef_step
                                   0.0)         # jump_threshold
group.execute(plan, wait=True)